# Test for å lage kontroller av Finansiering av investeringene

## Krav for at tabellen skal være korrekt
1. Andel av brutto investeringsutgifter skal være 100 % per kommune per årgang
2. Beløp summert per kommune per årgang til tilsvare brutto investeringsutgifter
3. Årlig utvikling skal være fornuftig 
    - Definisjon av fornuftig utvikling må bestemmes

### Steg
- Importerer tabeller fra Linux. Trenger to tabeller:
    - Finansiering av investeringene
    - Finansielle grunnlagsdata 
    
1. Kontroller om andel er 100 %
    - Ok med avvik på 1 %
    - Hvor mange kommuner har høyere avvik
        - Er dette mer/mindre enn tidligere årganger
        - Ta ut liste over kommuner med avvik
2. Kontroller om sum av investeringene tilsvarer brutto investeringsutgifter for kommunen
    - Hvor mange avviker
    - Hvor betydelig er avviket
    - Er det sammenheng mellom kommunene som avviker på kontroll 1 og 2
3. Kontroller den årlige utviklingen for alle tilgjengelige årganger
    - Prosentvis utvikling 
    - Plotte utviklingen
    - Se om HP-metoden fra Kostra biblioteket kan være et alternativ



In [ ]:
import pandas as pd
import numpy as np
import glob

## Importerer og ordner data

In [ ]:
#Sjekker hva som ligger i mappen på linux
#Har tidligere lagret test-data på egen Linux mappe
paths = glob.glob("/ssb/bruker/mtk/Testdata/publiseringstabeller til kontroll/*")
paths

In [ ]:
#Henter filen, sjekker innhold
fininv = pd.read_csv("/ssb/bruker/mtk/Testdata/publiseringstabeller til kontroll/finansiering_av_investeringene.csv", sep = ';')
fininv.info()
fininv.head()

In [ ]:
fingr = pd.read_csv('/ssb/bruker/mtk/Testdata/publiseringstabeller til kontroll/finansielle_grunnlagsdata.csv', sep = ';')
fingr.dtypes
fingr.info()

In [ ]:
#Retter datatyde
#fininv['andelinv'] = fininv['andelinv'].astype("float64")
#fininv.convert_dtypes().info()

## Kontroll 1: Sjekker andel av brutto investeringsutgifter

In [ ]:
#Filtrerer ut Halden og AGI23
#AGI23 = fininv[((fininv['kommuneregion'] == '3001')  | (fininv['kommuneregion'] == '0101')) & (fininv['art_kap'] == 'AGI23')]
#AGI23.head(10)

In [ ]:
#Grupperer finansiering av investeringene etter kommuneregion og periode
gruppert_alle = fininv.groupby(['kommuneregion', 'periode'])[['belop','andelinv']].sum()
gruppert_alle

In [ ]:
#Filtrerer ut observasjoner hvor andelinv avviker fra 100 med intervall på 0.5
diff = gruppert_alle[(gruppert_alle['andelinv'] >= 101.0) | (gruppert_alle['andelinv'] <= 99.0)].drop(columns=['belop'])
#diff.head(20)

In [ ]:
#Teller opp antall observasjoner hvor andel investering avviker per årgang
antall=diff.groupby('periode').count()
antall

### I det neste steget jukser jeg for å gjøre om det grupperte datasettet. Dermed får jeg en fil med periode, region og andel kun for kommunene med differanse
- Dette steget trenger jeg egentlig ikke siden jeg teller opp antall observasjoner over. 
- Lar det stå da jeg vil huske metoden

In [ ]:
#En merge ser ut til å løse groupby problemet. Det er riktignok juks...
juks=fingr.drop(columns=['art_kap', 'belop']).drop_duplicates(subset=['periode','kommuneregion'], keep= 'first')
#juks
test=pd.merge(diff, juks, on=['periode','kommuneregion'])
test

In [ ]:
diff_andel_2021=test[(test['periode'] == 2021)].sort_values(by=['andelinv'])
diff_andel_2021.to_csv("data/andel_summert_ulik100.csv")

### De to neste stegene grupperer, filtrerer og viser antall for en årgang. 
Mål: Hvordan iterere over alle årganger?
- Trengs ikke - har løst problemet på en annen måte over.
- Lar det likevel stå da det kan være lurt å finne ut hvordan jeg kan itterere over på et senere tidspunkt

In [ ]:
#Filtrerer ut 2021
p2021 = fininv[(fininv['periode'] == 2021)]
#Grupperer finansiering av investeringene etter kommuneregion og periode
gruppert_2021 = p2021.groupby(['kommuneregion', 'periode'])[['belop','andelinv']].sum()
#Filtrerer ut observasjoner hvor andelinv avviker fra 100 med intervall på 1.0
diff2021 = gruppert_2021[(gruppert_2021['andelinv'] >= 101.0) | (gruppert_2021['andelinv'] <= 99.0)]

#2020
p2020 = fininv[(fininv['periode'] == 2020)]
gruppert_2020 = p2020.groupby(['kommuneregion', 'periode'])[['belop','andelinv']].sum()
diff2020= gruppert_2020[(gruppert_2020['andelinv'] >= 101.0) | (gruppert_2020['andelinv'] <= 99.0)]

print('Antall kommuner som i 2021 hadde avvik på mer enn 1 %:', len(diff2021))
print('Antall kommuner som i 2020 hadde avvik på mer enn 1 %:', len(diff2020))

In [ ]:
p2020_test=fininv.loc[(fininv['periode']==2020) & (fininv['kommuneregion']=='0301'),'andelinv'].sum()
p2020_test

## Kontroll 2: Sjekker om sum beløp stemmer overens med brutto investeringutgifter

In [ ]:
#gruppert_alle.info
#fingr.head()

In [ ]:
biu = fingr[(fingr['art_kap']=='AGI1')]
#biu

In [ ]:
merged=biu.merge(gruppert_alle, on=['periode', 'kommuneregion']).rename(columns={'belop_x':'biu','belop_y':'summert'}).drop(columns=['andelinv'])
#merged

In [ ]:
merged['differanse'] = merged['summert'] - merged['biu']
#merged

In [ ]:
diff_tot=merged[(merged['differanse'] > 10) | (merged['differanse'] < -10)]
#diff_tot

In [ ]:
antall_tot=diff_tot.groupby('periode').count().drop(columns=['art_kap','biu','summert','differanse'])
antall_tot

In [ ]:
d2021=diff_tot[(diff_tot['periode'] == 2021)].sort_values(by=['differanse'])
d2021.to_csv("data/differanse_BIU_vs_fininv_summert.csv")

## Kontroll 3: Årlig utvikling

In [ ]:
import matplotlib.pyplot as plt
from matplotlib.pyplot import figure

In [ ]:
#merged['per_diff'] = (merged['biu']/merged['summert'].shift(1)-1).fillna(0)
#res = merged[(merged['per_diff'] > 1) | (merged['per_diff'] < -1)]
#res

In [ ]:
utv_land_test = fininv[(fininv['kommuneregion']=='EAK')&(fininv['art_kap']=='AGI29')].drop(columns=['andelinv']).sort_values(by=['art_kap','periode'])
utv_land_test

In [ ]:
utv_land_test['pct_change']=utv_land['belop'].pct_change()
utv_land_test

In [ ]:
utv_land_test.plot(x='periode', y='pct_change', kind= 'line')

In [ ]:
landet = fininv[(fininv['kommuneregion']=='EAK')].drop(columns=['andelinv']).sort_values(by=['art_kap','periode'])
landet

In [ ]:
pivoted=landet.pivot(index=['periode'], columns='art_kap', values = 'belop').reset_index()
pivoted = pivoted.rename_axis(None, axis=1)
pivoted.set_index('periode',inplace=True)
pivoted.columns
pivoted

In [ ]:
p=pivoted.pct_change().reset_index()
fjerner = p[(p['periode']!=2015)]
fjerner
#pivoted['pct_970']=pivoted['970'].pct_change()
#pivoted['pct_A670']=pivoted['A670'].pct_change()
#pivoted['pct_AGI23']=pivoted['AGI23'].pct_change()
#pivoted['pct_AGI26']=pivoted['AGI26'].pct_change()
#pivoted.drop(columns=['970', 'A670', 'AGI23', 'AGI26'])

In [ ]:
plt.plot('periode', '970', data=fjerner, marker='', color='blue', linewidth=1, label='970')
plt.plot('periode', 'A670', data=fjerner, marker='', color='red', linewidth=1, label='A670')
plt.plot('periode', 'AGI23', data=fjerner, marker='', color='green', linewidth=1, label='AGI23')
plt.plot('periode', 'AGI26', data=fjerner, marker='', color='yellow', linewidth=1, label='AGI26')
plt.plot('periode', 'AGI28', data=fjerner, marker='', color='brown', linewidth=1, label='AGI28')
plt.plot('periode', 'AGI29', data=fjerner, marker='', color='black', linewidth=1, label='AGI29')
plt.plot('periode', 'AGI31', data=fjerner, marker='', color='blue',linestyle='dashed', linewidth=1, label='AGI31')
plt.plot('periode', 'AGI4', data=fjerner, marker='', color='red',linestyle='dashed', linewidth=1, label='AGI4')
plt.plot('periode', 'AGI6', data=fjerner, marker='', color='green',linestyle='dashed', linewidth=1, label='AGI6')
plt.plot('periode', 'AGI7', data=fjerner, marker='', color='yellow',linestyle='dashed', linewidth=1, label='AGI7')
plt.plot('periode', 'AI660', data=fjerner, marker='', color='brown',linestyle='dashed', linewidth=1, label='AI660')
plt.plot('periode', 'AI729', data=fjerner, marker='', color='black',linestyle='dashed', linewidth=1, label='AI729')
plt.legend()
plt.figure(figsize=(10, 4))
plt.show()
plt.savefig('prosentvis_endring.jpg')